In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [7]:
movies=pd.read_csv('Boltzmann_Machines/ml-1m/movies.dat',sep='::',
                  header=None,engine='python',
                  encoding='latin-1')

In [9]:
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
users=pd.read_csv('Boltzmann_Machines/ml-1m/users.dat',sep='::',
                  header=None,engine='python',
                  encoding='latin-1')

In [11]:
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [12]:
ratings=pd.read_csv('Boltzmann_Machines/ml-1m/ratings.dat',sep='::',
                  header=None,engine='python',
                  encoding='latin-1')

In [13]:
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [16]:
movies.loc[357]

0                              361
1    It Could Happen to You (1994)
2                    Drama|Romance
Name: 357, dtype: object

In [22]:
training_set=pd.read_csv('Boltzmann_Machines/ml-100k/u1.base',delimiter='\t')

In [23]:
training_set

,1,1.1,5,874965758
0,1,2,3,876893171
1,1,3,4,878542960
2,1,4,3,876893119
3,1,5,3,889751712
4,1,7,4,875071561
...,...,...,...,...
79994,943,1067,2,875501756
79995,943,1074,4,888640250
79996,943,1188,3,888640250
79997,943,1228,3,888640275


In [25]:
training_set=np.array(training_set,dtype=int)

In [26]:
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [27]:
test_set=pd.read_csv('Boltzmann_Machines/ml-100k/u1.test',delimiter='\t')

In [29]:
test_set=np.array(test_set,dtype='int')

In [30]:
test_set

array([[        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       [        1,        14,         5, 874965706],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])